In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing Libraries

In [2]:
%load_ext tensorboard

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.image as mpimg
import random
import numpy as np

## Creating Functionalities

In [4]:
import os

#Walkthrough
for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/Colab Data_Files/Thakur_Data"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}.")

There are 5 directories and 0 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data.
There are 2 directories and 0 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data/Apple.
There are 4 directories and 0 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data/Apple/Train.
There are 0 directories and 570 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data/Apple/Train/Apple___Apple_scab.
There are 0 directories and 561 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data/Apple/Train/Apple___Black_rot.
There are 0 directories and 239 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data/Apple/Train/Apple___Cedar_apple_rust.
There are 0 directories and 1531 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data/Apple/Train/Apple___healthy.
There are 4 directories and 0 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data/Apple/Test.
There are 0 directories and 36 images in /content/drive/MyDrive/Colab Data_Files/Thakur_Data

In [5]:
def Walkthrough(data):
  for dirpath, dirnames, filenames in os.walk(f"/content/drive/MyDrive/Colab Data_Files/Thakur_Data/{data}"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}.")

In [6]:
def veiw_random_image(target_dir,target_class):
  #Set up target directory(We'll veiw images from here)
  target_folder = target_dir + target_class

  #Get a random image path
  random_image = random.sample(os.listdir(target_folder),1)
  print(random_image)
  print(random_image[0])
  #Read in the image and plot it using matplotlib
  img = mpimg.imread(target_folder + "/" +random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");

  print(f"Image Shape: {img.shape}") #Show the shape of image

  return img

In [7]:
def Veiw_batched_images(batch_Data,class_names):
  plt.figure(figsize=(10, 10))
  for image_batch, labels_batch in batch_Data:
      for i in range(12):
          ax = plt.subplot(3, 4, i + 1)
          plt.imshow(image_batch[i].numpy().astype("uint8"))
          plt.title(class_names[labels_batch[i]])
          plt.axis("off")

In [8]:
import datetime

#Create a Function to build a TensorBoard CallBack
def create_tensorboard_callback():
  #Create a Log Directory to store Logs
  logdir = os.path.join("/content/drive/MyDrive/Colab Data_Files/Thakur_Data/Log",
                        #Track The Logs whenever we run expirement i.e. joining it with datetime
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  
  return tf.keras.callbacks.TensorBoard(logdir)

## Data Augmentation

In [9]:
def Data_Augmentation(data_path,IMG_SIZE = (256,256),BATCH_SIZE = 32):
   # Set the seed
  tf.random.set_seed(42)

  # Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
  train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=5,
                                    width_shift_range=0.3,
                                    height_shift_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    channel_shift_range=0.4)
  
  valid_datagen = ImageDataGenerator(rescale=1./255)

  # Setup the train directories
  train_dir = '/content/drive/MyDrive/Colab Data_Files/Thakur_Data/' + data_path + '/Train'
  test_dir = '/content/drive/MyDrive/Colab Data_Files/Thakur_Data/' + data_path + '/Test'

  # Import data from directories and turn it into batches
  train_data = train_datagen.flow_from_directory(train_dir,
                                                batch_size=BATCH_SIZE, # number of images to process at a time 
                                                target_size=IMG_SIZE, # convert all images to be 224 x 224
                                                class_mode="categorical", # type of problem we're working on
                                                seed=42)
  
  valid_data = valid_datagen.flow_from_directory(test_dir,
                                                batch_size=BATCH_SIZE, # number of images to process at a time 
                                                target_size=IMG_SIZE, # convert all images to be 224 x 224
                                                class_mode="categorical", # type of problem we're working on
                                                seed=42)

  return train_data,valid_data

# Apple

In [9]:
train_Apple,val_Apple=Data_Augmentation('Apple')
train_Apple,val_Apple

Found 2901 images belonging to 4 classes.
Found 276 images belonging to 4 classes.


(<keras.preprocessing.image.DirectoryIterator at 0x7fca82ea77f0>,
 <keras.preprocessing.image.DirectoryIterator at 0x7fca82ea7a00>)

In [10]:
BATCH_SIZE  = 32
IMAGE_SIZE = 256
CHANNELS = 3
input_shape = (IMAGE_SIZE , IMAGE_SIZE , CHANNELS)
n_classes = 4

model_Apple = models.Sequential([
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model_Apple.build(input_shape=input_shape)

In [11]:
model_Apple.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 64)       0

In [12]:
model_Apple.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
history_1 = model_Apple.fit(train_Apple,
                        epochs=5,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        steps_per_epoch=len(train_Apple),
                        validation_data=val_Apple,
                        validation_steps=len(val_Apple))

Epoch 1/5
91/91 [==============================] - 756s 8s/step - loss: 1.1699 - accuracy: 0.5274 - val_loss: 1.2899 - val_accuracy: 0.4348
Epoch 2/5
91/91 [==============================] - 48s 532ms/step - loss: 0.9122 - accuracy: 0.5777 - val_loss: 0.8776 - val_accuracy: 0.6413
Epoch 3/5
91/91 [==============================] - 53s 584ms/step - loss: 0.7435 - accuracy: 0.6774 - val_loss: 0.7167 - val_accuracy: 0.7464
Epoch 4/5
91/91 [==============================] - 49s 540ms/step - loss: 0.6481 - accuracy: 0.7280 - val_loss: 0.5956 - val_accuracy: 0.7790
Epoch 5/5
91/91 [==============================] - 49s 537ms/step - loss: 0.5400 - accuracy: 0.7873 - val_loss: 0.5748 - val_accuracy: 0.7899


In [18]:
%tensorboard --logdir /content/drive/MyDrive/Colab Data_Files/Thakur_Data/Log

UsageError: Line magic function `%tensorboard` not found.


#**Corn**

In [10]:
train_Corn,val_Corn = Data_Augmentation('Corn')

Found 3473 images belonging to 4 classes.
Found 396 images belonging to 4 classes.


In [12]:
BATCH_SIZE  = 32
IMAGE_SIZE = 256
CHANNELS = 3
input_shape = (IMAGE_SIZE , IMAGE_SIZE , CHANNELS)
n_classes = 4

model_Corn = models.Sequential([
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.DepthwiseConv2D(kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model_Corn.build(input_shape=input_shape)

In [13]:
model_Corn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 depthwise_conv2d_1 (Depthwi  (None, 252, 252, 32)     320       
 seConv2D)                                                       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 126, 126, 32)     0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 124, 124, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                      

In [14]:
model_Corn.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
tensorboard = create_tensorboard_callback()

history_1 = model_Corn.fit(train_Corn,
                        epochs=10,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        callbacks=[tensorboard],
                        steps_per_epoch=len(train_Corn),
                        validation_data=val_Corn,
                        validation_steps=len(val_Corn))

Epoch 1/10
109/109 [==============================] - 785s 7s/step - loss: 1.0923 - accuracy: 0.4794 - val_loss: 0.8429 - val_accuracy: 0.5707
Epoch 2/10
109/109 [==============================] - 64s 585ms/step - loss: 0.7831 - accuracy: 0.6041 - val_loss: 0.7847 - val_accuracy: 0.5758
Epoch 3/10
109/109 [==============================] - 64s 585ms/step - loss: 0.7507 - accuracy: 0.6070 - val_loss: 0.7783 - val_accuracy: 0.5758
Epoch 4/10
109/109 [==============================] - 66s 603ms/step - loss: 0.7529 - accuracy: 0.6090 - val_loss: 0.7741 - val_accuracy: 0.5758
Epoch 5/10
109/109 [==============================] - 64s 586ms/step - loss: 0.7550 - accuracy: 0.6050 - val_loss: 0.7836 - val_accuracy: 0.5758
Epoch 6/10
109/109 [==============================] - 65s 591ms/step - loss: 0.7524 - accuracy: 0.6021 - val_loss: 0.7835 - val_accuracy: 0.5758
Epoch 7/10
109/109 [==============================] - 64s 586ms/step - loss: 0.7456 - accuracy: 0.5992 - val_loss: 0.7779 - val_accu

In [21]:
model_Corn_1 = models.Sequential([
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model_Corn_1.build(input_shape=input_shape)

In [22]:
model_Corn_1.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
tensorboard = create_tensorboard_callback()

history_corn = model_Corn_1.fit(train_Corn,
                        epochs=50,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        callbacks=[tensorboard],
                        steps_per_epoch=len(train_Corn),
                        validation_data=val_Corn,
                        validation_steps=len(val_Corn))

Epoch 1/50
109/109 [==============================] - 59s 539ms/step - loss: 0.6594 - accuracy: 0.7121 - val_loss: 0.2963 - val_accuracy: 0.8535
Epoch 2/50
109/109 [==============================] - 56s 511ms/step - loss: 0.3397 - accuracy: 0.8422 - val_loss: 0.6269 - val_accuracy: 0.7626
Epoch 3/50
109/109 [==============================] - 56s 510ms/step - loss: 0.2991 - accuracy: 0.8534 - val_loss: 0.2640 - val_accuracy: 0.8409
Epoch 4/50
109/109 [==============================] - 56s 509ms/step - loss: 0.3233 - accuracy: 0.8514 - val_loss: 0.2112 - val_accuracy: 0.9167
Epoch 5/50
109/109 [==============================] - 56s 511ms/step - loss: 0.2560 - accuracy: 0.8727 - val_loss: 0.2149 - val_accuracy: 0.9268
Epoch 6/50
109/109 [==============================] - 58s 534ms/step - loss: 0.2609 - accuracy: 0.8750 - val_loss: 0.1717 - val_accuracy: 0.9369
Epoch 7/50
109/109 [==============================] - 56s 509ms/step - loss: 0.2733 - accuracy: 0.8727 - val_loss: 0.2250 - val_ac

In [24]:
model_Corn_1.save('model.h5')

# **Grape**